# Creating PDFs - a simple introduction

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import zfit

assert tf.executing_eagerly() == False
print("TensorFlow version:", tf.__version__)

TensorFlow version: 1.12.0


## Create the data
To store and feed the data, a Dataset is the most comfortable way. While usually files can be used, here we use a numpy generated sample for convenience.

Note here, that while the numpy code gets executed, everything else (the explicit converion, the loading of the file if we had one) is *not yet* executed but a graph is built.

In [2]:
mu_true = 1.
sigma_true = 3.52
np_data = np.random.normal(mu_true, sigma_true, size=10000)
dataset = tf.data.Dataset.from_tensors(tf.cast(np_data, dtype=tf.float64))

# TODO: move the part below (maybe) inside out Dataset or something similar.
data_iterator = dataset.make_one_shot_iterator()  # equal to: no chunksizge; give me the dataset as a whole
data = data_iterator.get_next()  # next junk in a one-chunk iterator is trivial

/home/jonas/anaconda3/envs/zfit36/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [3]:
# HACK
data = tf.convert_to_tensor(np_data)
# HACK END

## Create the PDF
There are two ways of creating a pdf: use the provided ones or either define your own (see later/other examples). Here we gonna do the first approach.

The parameters follow the RooFit convention with  
`Parameter(name, initial_value, lower_limit (optional), upper_limit (optional))`

In [4]:
mu = zfit.Parameter("mu", 2.4, -1., 5.)
sigma = zfit.Parameter("sigma", 1.3, -1., 5.)

Let's define our own gaussian (currently unnormalized) pdf

In [5]:
class MyGauss(zfit.pdf.BasePDF):
    def __init__(self, loc, scale, name="my_first_gauss", **kwargs):  # any parameter names can be specified
        super().__init__(loc=loc, scale=scale, name=name, **kwargs)
        
    def _unnormalized_prob(self, x):
        loc = self.parameters['loc']
        scale = self.parameters['scale']
        return tf.exp(-0.5 * (loc - x) ** 2 / (scale ** 2))  # for example...


... and let's use it or stick to the already implemented one

In [6]:
use_our_own_gauss = False
if use_our_own_gauss:
    gauss1 = MyGauss(loc=mu, scale=sigma)
else:
    gauss1 = zfit.pdf.Gauss(mu=mu, sigma=sigma)

The pdf 'gauss1' contains all the usefull functions associated with a pdf.
It is important to note that the normalization range (`norm_range`) for the function always has to be specified *explicitely* (either in the function call, with a context manager or ...)

Let's check out the probability:

In [7]:
small_constants = [1., 3., 5.]
probs = gauss1.pdf(tf.constant(small_constants, dtype=tf.float64), norm_range=(-3, 5.1))
print("TensorFlow graph for probs", probs)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
TensorFlow graph for probs Tensor("Gauss/model/truediv:0", shape=(3,), dtype=float64)


(note here that we did *not yet* get our numerical result, but only a runnable graph)

Let's run our first graph and have a look at the probapilities

In [8]:
sess = tf.Session()
init = tf.global_variables_initializer()  # TODO: make more comfortable this step
sess.run(init)  # TODO: make more comfortable this step
result = sess.run(probs)
print("x values: {}\nresult:   {}".format(small_constants, result))
# Notice here that prob for 1., 3. are the same as sigma is 2.

x values: [1.0, 3.0, 5.0]
result:   [0.17515477 0.28119342 0.04233235]


## Fitting and NLL
As we have our probability at hand, we can use it for a loss function

In [10]:
nll = zfit.loss.UnbinnedNLL(gauss1, data, fit_range=(-5., 5.))

In [11]:
from zfit.minimize import MinuitMinimizer
minimizer = MinuitMinimizer(loss=nll)  # from TensorFlow

In [12]:
print("Minimizer created, start minimizing")

Minimizer created, start minimizing


In [13]:
minimium = minimizer.minimize(sess=sess, params=[mu, sigma])
print("Minimum:", minimium)

/home/jonas/Documents/physics/software/zfit_project/zfit_repo/zfit/minimizers/minimizer_minuit.py:61: InitialParamWarning: errordef is not given. Default to 1.
  **error_limit_kwargs)


FCN = 18722.354126996208 
 TOTAL NCALL = 92 
 NCALLS = 87 
 
 
 EDM = 0.10401229033810086 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 False 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 True 
 
 False

+,Name,Value,Hesse Error,Minos Error-,Minos Error+,Limit-,Limit+,Fixed?
0,mu:0,1.14979,0.102625,,,-1,5,No
1,sigma:0,4.14669,3.89692e-06,,,-1,5,No


Minimum: <zfit.minimizers.state.MinimizerState object at 0x7fc654379518>


In [14]:
minimium

In [15]:
mu_val, sigma_val = sess.run([mu.value(), sigma.value()])
print("Fit:  mu = {:.4f}, sigma = {:.4f}".format(mu_val, sigma_val))
print("True: mu = {:.4f}, sigma = {:.4f}".format(mu_true, sigma_true))

Fit:  mu = 1.1498, sigma = 4.1467
True: mu = 1.0000, sigma = 3.5200


In [15]:
#stop here  # prevents session close (this raises error with invalid syntax)

In [16]:
#sess.close()

## Sampling
An automatic accept-reject sampling (with garanteed no bias and linear speed in n_draws) is implemented. Let's call it:

In [16]:
sample_gauss = gauss1.sample(n=int(1e4), limits=(-10, 10))
sample_gauss_np = sess.run(sample_gauss)  

In [17]:
print("Number of generated samples:", len(sample_gauss_np))
#_ = plt.hist(sample_gauss_np, bins=30)

Number of generated samples: 1


In [18]:
plt.show()

## Integrate
Integration is also implemented. This is either normalized if a `norm_range` is specified or unnormalized if `norm_range` is False.

In [19]:
integral_normalized = gauss1.integrate(limits=(-20, 20), norm_range=(-20, 20))
integral_normalized_np = sess.run(integral_normalized)
print("Normalized integral over whole normalization range (should be 1)", integral_normalized_np)

Normalized integral over whole normalization range (should be 1) 1.0000181288236485


In [20]:
integral_unnormalized = gauss1.integrate(limits=(-20, 20), norm_range=False)
integral_unnormalized_np = sess.run(integral_unnormalized)
print("Unnormalized integral",
      integral_unnormalized_np)

Unnormalized integral 10.394136746759294


In [21]:
integral_normalized_small = gauss1.integrate(limits=(-1, 2), norm_range=(-5, 7))
integral_normalized_small_np = sess.run(integral_normalized_small)
print("normalized integral (expresses fraction)",
      integral_normalized_small_np)

normalized integral (expresses fraction) 0.32769682493292424


## Extend PDF
Extending a pdf and changing the probability to a number probability can be done by setting the yield of the pdf.
This does not only affect the `prob` function, but also any `integral` (given that a `norm_range` is specified! Otherwise, the yield is not applied)

In [23]:
yield1 = zfit.Parameter('yield', 42., 0., 100.)

# needed to initialize the parameters TODO
init = tf.global_variables_initializer()
sess.run(init)  

# set the yield
print("Gauss is extended:", gauss1.is_extended)
gauss1.set_yield(yield1)  # using `None` means to unextendid again
print("Gauss is extended:", gauss1.is_extended)


Gauss is extended: True
Gauss is extended: True


Doing the same integral as before:

In [24]:
integral_normalized_small = gauss1.integrate(limits=(-1, 2), norm_range=(-5, 7))
integral_normalized_small_np = sess.run(integral_normalized_small)
print("normalized integral (expresses fraction)",
      integral_normalized_small_np)

normalized integral (expresses fraction) 15.740840656376909


## Building advanced pdfs

In [25]:
gauss1 = zfit.pdf.Gauss(mu, sigma)
gauss2 = zfit.pdf.Gauss(mu, sigma)
gauss3 = zfit.pdf.Gauss(mu, sigma)
gauss4 = zfit.pdf.Gauss(mu=0.6, sigma=3.1)

param1 = zfit.Parameter('param1', 0.3)
param2 = zfit.Parameter('param1', 4.5)
yield1 = zfit.Parameter('param1', 42.)

In [26]:
composite_param = zfit.core.parameter.ComposedParameter(name='composite1', tensor=param1 * yield1 **2 * 4.)
gauss_comp_2var = zfit.pdf.Gauss(mu=param1, sigma=composite_param)
gauss_comp_3var = zfit.pdf.Gauss(mu=param2, sigma=composite_param)

In [27]:
composite_param.get_dependents()

{<tf.Variable 'param1:0' shape=() dtype=float64>,
 <tf.Variable 'param1_2:0' shape=() dtype=float64>}

In [28]:
gauss_comp_2var.get_dependents()

{<tf.Variable 'param1:0' shape=() dtype=float64>,
 <tf.Variable 'param1_2:0' shape=() dtype=float64>}

In [29]:
gauss_comp_3var.get_dependents()

{<tf.Variable 'param1:0' shape=() dtype=float64>,
 <tf.Variable 'param1_1:0' shape=() dtype=float64>,
 <tf.Variable 'param1_2:0' shape=() dtype=float64>}

In [30]:
gauss_sum_frac = param1 * gauss1 + gauss2
gauss_sum_frac = param1 * gauss1 + param2 * gauss2 + gauss3
gauss_sum_frac = ((param1 * gauss1) + (param2 * gauss2)) + gauss3
gauss_sum_frac = ((param1 * gauss1) + gauss3) + (param2 * gauss2)  # TODO: does it work?
#
# or
# gauss_sum_frac = zfit.pdf.SumPDF(pdfs=[gauss1, gauss2], fracs=[param1])

gauss_sum_extended = yield1 * gauss3 + 55. * gauss4
#
# or
# gauss3.set_yield(yield1), gauss4.set_yield(55.)
# gauss_sum_frac = zfit.pdf.SumPDF(pdfs=[gauss3, gauss4])
#
# or
# gauss_sum_frac = zfit.pdf.SumPDF(pdfs=[gauss1, gauss2], fracs=[0.3])
#
# or
# gauss_sum_frac = zfit.pdf.SumPDF(pdfs=[gauss1, gauss2], fracs=[yield1, 55.])

In [31]:
gauss_sum_frac.is_extended

False

In [32]:
gauss_sum_extended.is_extended

True

## Functions

In [33]:
from zfit.models import functions
func1 = functions.SimpleFunction(func=lambda x: x**2, n_dims=1)
func2 = functions.SimpleFunction(func=lambda x: x**4, n_dims=1)
func3 = functions.SimpleFunction(func=lambda x: x**6, n_dims=1)

In [34]:
func4 = param1 * func1

In [35]:
func5 = func4 + 3* func2 * func1

In [36]:
func6 = gauss2.as_func(norm_range=False)

In [37]:
pdf1 = func5.as_pdf()

AssertionError: kwargs not empty, the following arguments are not captured: {'FIXED_autoparam_3:0': <tf.Variable 'FIXED_autoparam_3:0' shape=() dtype=float64>, 'param1:0': <tf.Variable 'param1:0' shape=() dtype=float64>}